# Reto: Análisis de Uso de TransMilenio en Domingos
### *by Juan Pablo Morales*

In [2]:
import pandas as pd

In [ ]:
# TODO: Remove special chars from Valor column so that conversion works properly
def optimize_data_frame(df: pd.DataFrame):
    """This function will do the following optimizations to the provided data-frame `df`:
        - Set columns for date and numeric values to the right type
        - Remove useless columns/rows
        - Optimize cols with few unique values using `Series.astype()` method with `"category"` as the `dtype` parameter, significantly reducing memory consumption.

    Args:
        df (pd.DataFrame): data frame to be optimized and cleaned 

    Returns:
        pd.DataFrame: optimized copy of the original data-frame
    """
    optimized_df = df.copy()
    # Set columns for date and numeric values to the right type
    optimized_df["Fecha_Transaccion"] = pd.to_datetime(df["Fecha_Transaccion"], format="%Y-%m-%d %H:%M:%S")

    # Remove useless columns/rows
    # TODO

    # Optimize cols with few unique values using `Series.astype()` method with `"category"` as the `dtype` parameter, significantly reducing memory consumption.
    cols_to_categorize = [
        "Acceso_Estacion",
        "Day_Group_Type",
        "Dispositivo",
        "Emisor",
        "Estacion_Parada",
        "Fase",
        "Hora_Pico_SN",
        "ID_Vehiculo",
        "Linea",
        "Nombre_Perfil",
        "Numero_Tarjeta",
        "Operador",
        "Ruta",
        "Sistema",
        "Tipo_Tarifa",
        "Tipo_Tarjeta",
        "Tipo_Vehiculo"
    ]
    optimized_df[cols_to_categorize] = df[cols_to_categorize].astype("category")

    return optimized_df

In [ ]:
def collect_data(data_list):
    """Method that groups all the data into a single data-frame

    Args:
        data_list (list): list of paths of csv files from which data is to be collected and grouped
    """
    final_df = pd.DataFrame()

    # for data in data_list: 
    #     return pd.read_csv(data, parse_dates=["Fecha_Clearing"], date_format="%Y-%m-%d")
    final_df = pd.read_csv(data_list[0], parse_dates=["Fecha_Clearing"], date_format="%Y-%m-%d")
    return final_df

    

In [85]:
day1 = pd.read_csv("challenge/validacion20241020", parse_dates=["Fecha_Clearing"], date_format="%Y-%m-%d")

In [89]:
# Read info of the four days from csv files and create a dataframe for each one
# data_list = ["challenge/validacion20241020"]
# day1 = collect_data(data_list)

# day1.info()
optimize_data_frame(day1).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 638602 entries, 0 to 638601
Data columns (total 22 columns):
 #   Column                      Non-Null Count   Dtype         
---  ------                      --------------   -----         
 0   Acceso_Estacion             638602 non-null  category      
 1   Day_Group_Type              638602 non-null  category      
 2   Dispositivo                 638602 non-null  category      
 3   Emisor                      638602 non-null  category      
 4   Estacion_Parada             638602 non-null  category      
 5   Fase                        638602 non-null  category      
 6   Fecha_Clearing              638602 non-null  datetime64[ns]
 7   Fecha_Transaccion           638602 non-null  datetime64[ns]
 8   Hora_Pico_SN                638602 non-null  category      
 9   ID_Vehiculo                 0 non-null       category      
 10  Linea                       638602 non-null  category      
 11  Nombre_Perfil               638602 non-

In [57]:
day1.head()

,Acceso_Estacion,Day_Group_Type,Dispositivo,Emisor,Estacion_Parada,Fase,Fecha_Clearing,Fecha_Transaccion,Hora_Pico_SN,ID_Vehiculo,...,Numero_Tarjeta,Operador,Ruta,Saldo_Despues_Transaccion,Saldo_Previo_a_Transaccion,Sistema,Tipo_Tarifa,Tipo_Tarjeta,Tipo_Vehiculo,Valor
0,(LA) ALIMENTACION CASTILLA Y BIBLIOTECA TINTAL,Dia 2,10000784,3101000,(05100) Banderas P. Central,Fase 3,2024-10-20,2024-10-20 03:50:43,Peak Time,NaN,...,d970ef6f244d5c52ed0319c87dc31e2c0eea23ed151fc7...,(201) Trunk agency,NaN,1730.0,4680.0,TRONCAL,"1,0",tullave Plus,NaN,$ 2950.0
1,(01) Acceso Peatonal Norte,Dia 2,10001263,3101000,(07010) Bosa,Fase 3,2024-10-20,2024-10-20 03:53:34,Peak Time,NaN,...,a432a8fb2c4ed0eee674c0d3ae6758e27f02d71db785ff...,(201) Trunk agency,NaN,14750.0,17700.0,TRONCAL,"1,0",tullave Básica,NaN,$ 2950.0
2,(01) Acceso Peatonal Norte,Dia 2,10001264,3101000,(07010) Bosa,Fase 3,2024-10-20,2024-10-20 03:53:57,Peak Time,NaN,...,82e95c47d12e418f3d661df39905cbce80ef9e21804c53...,(201) Trunk agency,NaN,250.0,3200.0,TRONCAL,"1,0",tullave Básica,NaN,$ 2950.0
3,(BA) BATERIA VAGON ORIENTE PARQUE EL TUNAL,Dia 2,10000528,3101000,(08001) Parque el Tunal,Fase 3,2024-10-20,2024-10-20 03:54:20,Peak Time,NaN,...,f72f11a243e1821860d4525f6ba295de6f21fe783fa7e2...,(201) Trunk agency,NaN,68000.0,70950.0,TRONCAL,"1,0",tullave Básica,NaN,$ 2950.0
4,(23) Acceso Peatonal,Dia 2,40000160,3101000,(05000) Portal Américas,Fase 3,2024-10-20,2024-10-20 03:54:47,Peak Time,NaN,...,ff62cbf07e71e84f49e9f6af4671f60efb77c1735229ad...,(201) Trunk agency,NaN,54230.0,56730.0,TRONCAL,"1,0",tullave Plus,NaN,$ 2500.0


In [73]:
day1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 638602 entries, 0 to 638601
Data columns (total 22 columns):
 #   Column                      Non-Null Count   Dtype         
---  ------                      --------------   -----         
 0   Acceso_Estacion             638602 non-null  object        
 1   Day_Group_Type              638602 non-null  object        
 2   Dispositivo                 638602 non-null  int64         
 3   Emisor                      638602 non-null  int64         
 4   Estacion_Parada             638602 non-null  object        
 5   Fase                        638602 non-null  object        
 6   Fecha_Clearing              638602 non-null  datetime64[ns]
 7   Fecha_Transaccion           638602 non-null  datetime64[ns]
 8   Hora_Pico_SN                638602 non-null  object        
 9   ID_Vehiculo                 0 non-null       float64       
 10  Linea                       638602 non-null  object        
 11  Nombre_Perfil               638602 non-

In [18]:
day1.nunique()

Acceso_Estacion                  401
Day_Group_Type                     2
Dispositivo                     1371
Emisor                             8
Estacion_Parada                  147
Fase                               1
Fecha_Clearing                     1
Fecha_Transaccion              64752
Hora_Pico_SN                       1
ID_Vehiculo                        0
Linea                             13
Nombre_Perfil                      7
Numero_Tarjeta                388048
Operador                           1
Ruta                               0
Saldo_Despues_Transaccion       8611
Saldo_Previo_a_Transaccion      8744
Sistema                            1
Tipo_Tarifa                        3
Tipo_Tarjeta                       2
Tipo_Vehiculo                      0
Valor                              3
dtype: int64

In [91]:
optimize_data_frame(day1).dtypes

Acceso_Estacion                     category
Day_Group_Type                      category
Dispositivo                         category
Emisor                              category
Estacion_Parada                     category
Fase                                category
Fecha_Clearing                datetime64[ns]
Fecha_Transaccion             datetime64[ns]
Hora_Pico_SN                        category
ID_Vehiculo                         category
Linea                               category
Nombre_Perfil                       category
Numero_Tarjeta                      category
Operador                            category
Ruta                                category
Saldo_Despues_Transaccion            float64
Saldo_Previo_a_Transaccion           float64
Sistema                             category
Tipo_Tarifa                         category
Tipo_Tarjeta                        category
Tipo_Vehiculo                       category
Valor                                 object
dtype: obj

In [7]:
day1.columns

Index(['Acceso_Estacion', 'Day_Group_Type', 'Dispositivo', 'Emisor',
       'Estacion_Parada', 'Fase', 'Fecha_Clearing', 'Fecha_Transaccion',
       'Hora_Pico_SN', 'ID_Vehiculo', 'Linea', 'Nombre_Perfil',
       'Numero_Tarjeta', 'Operador', 'Ruta', 'Saldo_Despues_Transaccion',
       'Saldo_Previo_a_Transaccion', 'Sistema', 'Tipo_Tarifa', 'Tipo_Tarjeta',
       'Tipo_Vehiculo', 'Valor'],
      dtype='object')

In [9]:
day1.axes

[RangeIndex(start=0, stop=638602, step=1),
 Index(['Acceso_Estacion', 'Day_Group_Type', 'Dispositivo', 'Emisor',
        'Estacion_Parada', 'Fase', 'Fecha_Clearing', 'Fecha_Transaccion',
        'Hora_Pico_SN', 'ID_Vehiculo', 'Linea', 'Nombre_Perfil',
        'Numero_Tarjeta', 'Operador', 'Ruta', 'Saldo_Despues_Transaccion',
        'Saldo_Previo_a_Transaccion', 'Sistema', 'Tipo_Tarifa', 'Tipo_Tarjeta',
        'Tipo_Vehiculo', 'Valor'],
       dtype='object')]